In [1]:
import os
os.chdir('./Code')

In [2]:

from signal_store.signal_store_nc import SignalStoreNc
from pred_store.pred_store_nc import PredRetStoreNc

In [3]:
from core_models.huber_model import HuberRegressor

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
start_date = '20090101'
end_date = '20161231'

In [5]:
sig_dir = '../TechSig/'
ret_dir = '../Ret/'

sig_cfg = {'data_dir' : sig_dir,  }
ret_cfg = {'data_dir' : ret_dir, 'pred_horizon' : {'h0_1':[0,1], 'h2_5' : [2,6], 'h6_20' :[6,20] } }

In [7]:
region = 'US' 
freq = '1d'
timezone = 'EST' 
ukeys= None


In [7]:
sig_store_obj = SignalStoreNc(region, freq, timezone, ukeys,sig_cfg )
sig_store_obj.read_data(start_date, end_date)

ret_store_obj = PredRetStoreNc(region, freq, timezone, ukeys,ret_cfg )
ret_store_obj.read_data(start_date, end_date )

X = sig_store_obj.get_2d_data().to_pandas().T
Y = ret_store_obj.get_2d_data().to_pandas().T

In [8]:
import xarray as xr

In [9]:
sig_xr = sig_store_obj.get_full_data()
ret_xr = ret_store_obj.get_full_data()

sig_A = sig_xr.A.values
ret_A = ret_xr.A.values

In [10]:
sig_key = sig_A[0]
ret_key = ret_A[0]

In [11]:
x_df = sig_xr.loc[sig_key].to_pandas()
y_df = ret_xr.loc[ret_key].to_pandas()

In [12]:
x_df.corrwith(y_df, axis=1).mean()

-0.0004967538488177445

In [13]:
import pandas as pd

In [14]:
import os

In [15]:
stats_dir = '../Stats/'
if not os.path.exists(stats_dir) :
    os.makedirs(stats_dir)

In [16]:
stats_data = {}
window = 252

sig_df_dict = { sig_key:sig_xr.loc[sig_key].to_pandas() for sig_key in sig_A }

ret_df_dict = { ret_key:ret_xr.loc[ret_key].to_pandas() for ret_key in ret_A }

for sig_key in sig_A :
    x_df = sig_df_dict[sig_key]
    stats_df = pd.DataFrame()

    for ret_key in ret_A :
        y_df = ret_df_dict[ret_key]
        
        l = int(ret_key.split('_')[-1])
        
        y_vol = y_df.std(axis=1).rolling(window, min_periods = 22).mean().shift(l)
        x_vol = x_df.std(axis=1).rolling(window, min_periods = 22).mean().shift(l)
        ic = x_df.corrwith(y_df, axis=1).rolling(window, min_periods = 22).mean().shift(l)


        beta = y_vol * ic / x_vol

        
        stats_df[ret_key+'.ic'] = ic
        stats_df[ret_key+'.beta'] = beta
        
    stats_df = stats_df.iloc[window:,:]
        
    #stats_df.to_pickle(stats_dir +f'./{sig_key}.pkl')
    stats_data[sig_key] = stats_df
        

In [17]:
stats_xr = xr.Dataset(stats_data).to_array()

In [18]:
stats_xr = stats_xr.rename({'variable':'A','T':'T' ,'dim_1':'S', })

In [19]:
stats_xr

<xarray.DataArray (A: 83, T: 1006, S: 6)>
array([[[-1.18372700e-04, -1.70703897e-06,  4.20159383e-03,
          1.20302500e-04,  7.69515790e-03,  4.04272732e-04],
        [-4.78362860e-04, -6.89024110e-06,  4.15772850e-03,
          1.19081642e-04,  7.55066063e-03,  3.96378325e-04],
        [-6.37051098e-04, -9.16898810e-06,  4.11419260e-03,
          1.17944863e-04,  7.34992403e-03,  3.85720849e-04],
        ...,
        [ 6.40007617e-04,  7.80862909e-06,  3.19622995e-04,
          7.79463427e-06,  2.11089953e-03,  9.54639480e-05],
        [ 4.21836934e-04,  5.14530350e-06, -3.63780052e-03,
         -8.85576602e-05,  2.11932583e-03,  9.57801098e-05],
        [ 2.31974552e-04,  2.82269217e-06, -3.32437923e-03,
         -8.10493891e-05,  6.23027468e-03,  2.80631718e-04]],

       [[ 3.07725465e-04,  4.43767322e-06,  4.22458711e-03,
          1.20960858e-04,  1.29548591e-02,  6.80596335e-04],
        [ 1.01331805e-04,  1.45956266e-06,  4.41247911e-03,
          1.26377963e-04,  1.29992290e-02,  6.82405537e-04],
        [ 2.47428218e-04,  3.56120003e-06,  4.59476857e-03,
          1.31721921e-04,  1.30423752e-02,  6.84458233e-04],
...
        [-7.73685223e-03, -9.43960722e-05, -1.07844681e-02,
         -2.63000409e-04, -8.17590284e-03, -3.69749432e-04],
        [-9.37678885e-03, -1.14372207e-04, -1.48105114e-02,
         -3.60543175e-04, -8.17114918e-03, -3.69284182e-04],
        [-8.42985160e-03, -1.02575366e-04, -1.44802961e-02,
         -3.53034051e-04, -1.18217031e-02, -5.32487690e-04]],

       [[ 1.31926663e-04,  1.90249917e-06,  7.35021699e-03,
          2.10455736e-04,  2.64273953e-02,  1.38838941e-03],
        [-7.78633990e-05, -1.12152852e-06,  7.35716905e-03,
          2.10716935e-04,  2.60679635e-02,  1.36845988e-03],
        [-5.45115973e-05, -7.84577860e-07,  7.33824647e-03,
          2.10371414e-04,  2.58591452e-02,  1.35707686e-03],
        ...,
        [ 5.50710400e-03,  6.71912824e-05,  1.04164783e-02,
          2.54026275e-04,  1.77014807e-02,  8.00537026e-04],
        [ 5.21336010e-03,  6.35893110e-05,  6.66657543e-03,
          1.62289361e-04,  1.73380358e-02,  7.83569451e-04],
        [ 4.77708029e-03,  5.81280448e-05,  7.10885604e-03,
          1.73316099e-04,  2.11834229e-02,  9.54169866e-04]]])
Coordinates:
  * T        (T) datetime64[ns] 2011-01-03 2011-01-04 ... 2014-12-30 2014-12-31
  * S        (S) object 'h0_1.ic' 'h0_1.beta' ... 'h6_20.ic' 'h6_20.beta'
  * A        (A) <U25 'volume_adi' 'volume_obv' ... 'others_dlr' 'others_cr'

In [20]:
stats_dir = './Stats/agg_xr/'
if not os.path.exists(stats_dir) :
    os.makedirs(stats_dir)

In [21]:
dt = stats_xr['T'].values


In [22]:


def write_xr(data_xr, data_dir, i):
    f = data_dir + pd.to_datetime(i).strftime('%Y%m%d') +'.nc'
    data_xr.loc[:,[i],:].to_netcdf(f) 
    

for i in dt :
    write_xr(stats_xr, stats_dir, i)
 

In [8]:
from model_container.model_container import ModelContainer 

In [14]:
cfg = {}

cfg['region'] = region
cfg['freq'] = freq
cfg['timezone'] = timezone
cfg['ukeys'] = ukeys
cfg['sig_cfg'] = {'class' : 'signal_store.signal_store_nc.SignalStoreNc', 'args' :{'data_dir':'../TechSig/'} }
cfg['pred_cfg'] = {'class' : 'pred_store.pred_store_nc.PredRetStoreNc', 'args' :{'data_dir':'../Ret/'} }
cfg['stats_cfg'] = {'class' : 'signal_store.signal_store_nc.SignalStoreNc', 'args' :{'data_dir':'../Stats/'} }
cfg['model_cfg'] = {'class' : 'core_models.huber_model.HuberRegressor', 'args' :{'alpha':1.0} }

In [15]:
mco =  ModelContainer(cfg)

In [17]:
start_date = '20100101'
end_date = '20111231'

In [18]:
mco.fit_models(start_date, end_date)

AttributeError: 'ModelContainer' object has no attribute 'ret_obj'